In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown --id 10pA2sZxgjfROY1f7Opre2TaU5vsIWp2M # demand
# !gdown --id 1eRB2HAxeUR99NnGqt22A9-HvXujaAvmN # DroneCost
# !gdown --id 1XZeBqc_ObO8giW1_WhI9IqZVcolAmISU # Drone Path
!gdown --id 1PAUBDBkGG75-PcQGy4RQlXee0YuQRSTs # Parameters

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=10pA2sZxgjfROY1f7Opre2TaU5vsIWp2M 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1PAUBDBkGG75-PcQGy4RQlXee0YuQRSTs
To: /content/Parameters.csv
100% 1.44k/1.44k [00:00<00:00, 2.43MB/s]


In [ ]:
import math
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from collections import deque
import heapq
from collections import defaultdict
import random as rn

In [ ]:
max_step=100

directions=[[-1,0,0],[1,0,0],[0,-1,0],[0,1,0],[-1,-1,0],[1,-1,0],[-1,1,0],[1,1,0]]
def new_cmp_lt(self,a,b):
    return a[1]<b[1]
def check_nofly(obj):
  if len(no_fly_regions)==0:
    return False
  for zone in no_fly_regions:
    if zone.is_inside(obj)==True:
      return True
  return False
class vertex:
  def __init__(self,xx=0.0,yy=0.0,zz=0.0):
    self.cordinate=np.array([xx,yy,zz])

  def __lt__(self,other):
    return self.cordinate[2]<other.cordinate[2]
  
  def tupple(self):
    return (self.cordinate[0],self.cordinate[1],self.cordinate[2])

  def euc_distance(self,another_point):
    return np.sqrt(np.sum(np.square(np.subtract(self.cordinate,another_point.cordinate))))

  def is_valid(self):
    return self.cordinate[2]>=0

  def min_time(self,point,velocities,init_time=0):
    vx=velocities[0]
    vy=velocities[1]
    vz=velocities[2]
    cost={}
    parent={}
    g={}
    d={}
    max_h=-1e9
    h=0
    init_state=vertex(self.cordinate[0],self.cordinate[1],point.cordinate[2])
    h=max(0,point.cordinate[2]-self.cordinate[2])
    cost[init_state]=0
    parent[init_state]=(None,0)
    fin_state=None
    g[init_state]=0
    d[init_state]=0
    closed_list=set()
    pq = []
    heapq.heappush(pq, (0, init_state))
    ff=True
    while len(pq)>0 and ff:
      dis=pq[0][0]
      obj=pq[0][1]
      heapq.heappop(pq)
      if obj.tupple() in closed_list:
        continue
      # print(obj.cordinate)
      # max_h=max(max_h,obj.cordinate[2])
      closed_list.add(obj.tupple())
      if np.array_equal(obj.cordinate,point.cordinate):
        cost[obj]=dis
        fin_state=obj
        ff=False
        break
      for ind in directions:
        step=0
        if obj.euc_distance(point)>=10*max_step:
          step=max_step
        elif obj.euc_distance(point)>=10:
          step=int(obj.euc_distance(point)/10)
        else:
          step=1
        flag=True
        # while flag and step>0:
        new_obj=vertex(obj.cordinate[0]+ind[0]*step,obj.cordinate[1]+ind[1]*step,obj.cordinate[2]+ind[2]*step)
        
        if new_obj.tupple() not in closed_list and new_obj.is_valid() and not check_nofly(new_obj):
          
          temp=abs(ind[0])+abs(ind[1])+abs(ind[2])
          nstep=0
          if temp==1:
            nstep=1
            if (abs(ind[0])==1 or abs(ind[1])==1) :
              temp=(1*step)/vx
            else:
              temp=(1*step)/vz

          elif temp==2:
            nstep=math.sqrt(2)
            if abs(ind[0])==1 and abs(ind[1])==1:
              temp=(math.sqrt(2)*step)/vx
            else:
              temp=(math.sqrt(2)*step)/vx
          else:
            nstep=math.sqrt(3)
            temp=(math.sqrt(3)*step)/vx
          
          gnew=g[obj]+temp
          dnew=d[obj]+nstep*step
          hnew=new_obj.euc_distance(point)/vx
          fnew=gnew+hnew
          if new_obj not in cost or cost[new_obj]>fnew:
            parent[new_obj]=(obj,(nstep*step)/vx)
            cost[new_obj]=fnew
            g[new_obj]=gnew
            d[new_obj]=dnew
            heapq.heappush(pq,(fnew,new_obj))      
    path=[]
    time_stamp=init_time+g[obj]+h/vz
    pr_state=fin_state
    
    while pr_state is not None:
      path.append((pr_state,math.ceil(time_stamp)))
      a,b=parent[pr_state]
      pr_state=a
      time_stamp-=b
    path.append((self,math.ceil(time_stamp-h/vz)))
    return math.ceil(g[obj]+h/vz),d[obj],h,path   # time,distance,height

In [ ]:
class cuboid:
    def __init__(self,a1,a2,a3,a4,a5,a6,a7,a8):
        self.v1=a1
        self.v2=a2 
        self.v3=a3
        self.v4=a4
        self.v5=a5
        self.v6=a6
        self.v7=a7
        self.v8=a8
    def is_inside(self,point):
        u = np.subtract(self.v1.cordinate, self.v2.cordinate)
        v = np.subtract(self.v1.cordinate, self.v4.cordinate)
        w = np.subtract(self.v1.cordinate, self.v5.cordinate)
        if min(np.dot(u, self.v1.cordinate), np.dot(u, self.v2.cordinate)) < np.dot(u, point.cordinate) < max(np.dot(u, self.v1.cordinate), np.dot(u, self.v2.cordinate))\
        and min(np.dot(v, self.v1.cordinate), np.dot(v, self.v4.cordinate)) < np.dot(v, point.cordinate) < max(np.dot(v, self.v1.cordinate), np.dot(v, self.v4.cordinate))\
        and min(np.dot(w, self.v1.cordinate), np.dot(w, self.v5.cordinate)) < np.dot(w,point.cordinate) < max(np.dot(w, self.v1.cordinate), np.dot(w, self.v5.cordinate)):
            return True
        else :
            return False


In [ ]:
class Item:
    def __init__(self, idd, weight, l, b, h):
        self.id=idd
        self.weight=weight
        self.length = l
        self.breadth = b
        self.height = h
        self.volume = l*b*h


In [ ]:
class Warehouse:
    def __init__(self,idd,xx,yy,zz,amp):
        self.id=idd
        self.location=vertex(xx,yy,zz)
        self.items=[]
        self.amp=amp
        self.delivery_failure=0


In [ ]:
class City:
  def __init__(self,idd,xx,yy,zz,item,starttime,endtime,warehouse,day):
    self.id=idd
    self.item=item
    self.day = day
    self.starttime=starttime
    self.warehouse = warehouse
    self.endtime=endtime
    self.location=vertex(xx,yy,zz)
    self.delivery_failure = 0
  def __lt__(self,other):
    return self.endtime <other.endtime

In [ ]:
class Charging_station:
    def __init__(self,idd,xx,yy,zz,max_slots,amp,time_dict):
        self.id=idd
        self.location=vertex(xx,yy,zz)
        self.max_slots=max_slots
        self.amp=amp
        self.time_dict = []


In [ ]:
# pay_load_weight = [5,6,7,8,9,10]
# pay_load_vol = [200,500,1000,2000,3000,5000]
# base_weight = [2,2.5,3,3.5,4,5]
# battery_cap = [2000,2500,3000,4000,5000,10000]
# max_slots = [1,1,2,2,2,4]
# fixed_cost = [10,15,20,20,30,50]
# main_cost = [5,8,13,15,20,25]

In [ ]:
class Drone:
    def __init__(self,idd,M,P,Q,xx,yy,zz,A,B,C,py_vol,py_weight,bat_cap,fixed_cost,var_cost,max_slots,base_weight,num):
        self.id=idd
        self.M=M
        self.P=P
        self.Q=Q
        # self.f=f
        self.A=A
        self.B=B
        self.C=C
        self.initial=vertex(xx,yy,zz)
        self.fixed_cost=fixed_cost
        self.var_cost=var_cost
        self.payload_capacity_weight=py_weight
        self.payload_capacity_vol=py_vol
        self.base_weight=base_weight
        self.battery_capacity=bat_cap
        self.max_slots=max_slots
        self.number_of_drones=num

    def velocities(self,f=0):
        return [self.M-f*self.P,self.M-f*self.P,self.M-f*self.Q]


In [ ]:
class Location:
  def __init__(self,xx,yy,zz,loc_type, index):
    self.location = vertex(xx,yy,zz)
    self.loc_type = 0
    self.index = 0 
    #If type == 0, contains the warehouse number, if type == 1 contains the charging station number, if type==2 contains the city index

In [ ]:

class Curr_status:
    def __init__(self,droneinfo):
        self.drone = droneinfo
        self.curr_charge = self.drone.battery_capacity
        self.weight = 0
        self.volume = 0     
        self.payload = []
    # drone type and corresponding properties
    # current charge, weight, volume, cost??
    # list of items inside drone

In [ ]:
demand = pd.read_csv('Demand_Day1.csv')
demand['DeliveryFrom'] = pd.to_datetime(demand['DeliveryFrom'])
demand['DeliveryTo'] = pd.to_datetime(demand['DeliveryTo'])
# demand['Day']=[int(str(x)[-1]) for x in demand['Day']]
# for i in range(len(demand)):
#     demand.loc[i,'DelveryFrom'] = demand.loc[i,'DelveryFrom']+ timedelta(hours=24*int(demand.loc[i,'Day']))
earliest_time=pd.to_datetime('08:00:00')
demand['DeliveryFrom']=[(x-earliest_time).total_seconds() for x in demand['DeliveryFrom']]
demand['DeliveryTo']=[(x-earliest_time).total_seconds() for x in demand['DeliveryTo']]


In [ ]:
parameters = pd.read_csv('Parameters.csv')
print(parameters.to_string())

    Parameter_ID    Value   Unit           Type
0   MaxSpeed (M)    12.00    m/s            NaN
1        Cost(C)     1.20   $/AH            NaN
2            X11    40.00  Meter      Noflyzone
3            X12    40.00  Meter      Noflyzone
4            X13   500.00  Meter      Noflyzone
5            X14   500.00  Meter      Noflyzone
6            X15   500.00  Meter      Noflyzone
7            X16    40.00  Meter      Noflyzone
8            X17    40.00  Meter      Noflyzone
9            X18   500.00  Meter      Noflyzone
10           Y11   -50.00  Meter      Noflyzone
11           Y12   -50.00  Meter      Noflyzone
12           Y13   -50.00  Meter      Noflyzone
13           Y14   -50.00  Meter      Noflyzone
14           Y15   300.00  Meter      Noflyzone
15           Y16   300.00  Meter      Noflyzone
16           Y17   300.00  Meter      Noflyzone
17           Y18   300.00  Meter      Noflyzone
18           Z11     0.00  Meter      Noflyzone
19           Z12    60.00  Meter      No

In [ ]:
charging_stations = [] # list of charging locations
warehouses = [] # list of warehouses
cities = [] # list of order of cities
total_cities = []
no_fly_regions = []
Drones = []
items = [] 
# need to hardcode

In [ ]:
max_speed = parameters[parameters['Parameter_ID']=='MaxSpeed (M)'].Value.iloc[0]
cost = parameters[parameters['Parameter_ID']=='Cost(C)'].Value.iloc[0]
max_time = 12000

In [ ]:
for i in range(10):
    temp = parameters[parameters['Type']=='Noflyzone']

    if(len(temp[temp['Parameter_ID']=='X' + str(i+1) + '1'])==0):
        break
    
    x1 = temp[temp['Parameter_ID']=='X' + str(i+1) + '1']['Value'].iloc[0]
    x2 = temp[temp['Parameter_ID']=='X' + str(i+1) + '2']['Value'].iloc[0]
    x3 = temp[temp['Parameter_ID']=='X' + str(i+1) + '3']['Value'].iloc[0]
    x4 = temp[temp['Parameter_ID']=='X' + str(i+1) + '4']['Value'].iloc[0]
    x5 = temp[temp['Parameter_ID']=='X' + str(i+1) + '5']['Value'].iloc[0]
    x6 = temp[temp['Parameter_ID']=='X' + str(i+1) + '6']['Value'].iloc[0]
    x7 = temp[temp['Parameter_ID']=='X' + str(i+1) + '7']['Value'].iloc[0]  
    x8 = temp[temp['Parameter_ID']=='X' + str(i+1) + '8']['Value'].iloc[0]

    y1 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '1']['Value'].iloc[0]
    y2 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '2']['Value'].iloc[0]
    y3 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '3']['Value'].iloc[0]
    y4 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '4']['Value'].iloc[0]
    y5 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '5']['Value'].iloc[0]
    y6 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '6']['Value'].iloc[0]
    y7 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '7']['Value'].iloc[0]  
    y8 = temp[temp['Parameter_ID']=='Y' + str(i+1) + '8']['Value'].iloc[0]

    z1 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '1']['Value'].iloc[0]
    z2 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '2']['Value'].iloc[0]
    z3 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '3']['Value'].iloc[0]
    z4 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '4']['Value'].iloc[0]
    z5 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '5']['Value'].iloc[0]
    z6 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '6']['Value'].iloc[0]
    z7 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '7']['Value'].iloc[0]  
    z8 = temp[temp['Parameter_ID']=='Z' + str(i+1) + '8']['Value'].iloc[0]    
    a1 = vertex(x1,y1,z1)
    a2 = vertex(x2,y2,z2)
    a3 = vertex(x3,y3,z3)
    a4 = vertex(x4,y4,z4)
    a5 = vertex(x5,y5,z5)
    a6 = vertex(x6,y6,z6)
    a7 = vertex(x7,y7,z7)
    a8 = vertex(x8,y8,z8)
    nofly = cuboid(a1,a2,a3,a4,a5,a6,a7,a8)
    no_fly_regions.append(nofly)

In [ ]:
temp = parameters[parameters['Parameter_ID']=='WH1X']
if len(temp) != 0:
  x = parameters[parameters['Parameter_ID']=='WH1X'].iloc[0]['Value']
  y = parameters[parameters['Parameter_ID']=='WH1Y'].iloc[0]['Value']
  z = parameters[parameters['Parameter_ID']=='WH1Z'].iloc[0]['Value']
  print(x, y, z)
  
  warehouses.append(Warehouse(1, x,y,z, 5))

temp = parameters[parameters['Parameter_ID']=='WH2X']
if len(temp) != 0:

  x = parameters[parameters['Parameter_ID']=='WH2X'].iloc[0]['Value']
  y = parameters[parameters['Parameter_ID']=='WH2Y'].iloc[0]['Value']
  z = parameters[parameters['Parameter_ID']=='WH2Z'].iloc[0]['Value']

  warehouses.append(Warehouse(2, x,y,z, 5))

temp = parameters[parameters['Parameter_ID']=='WH3X']
if len(temp) != 0:

  x = parameters[parameters['Parameter_ID']=='WH3X'].iloc[0]['Value']
  y = parameters[parameters['Parameter_ID']=='WH3Y'].iloc[0]['Value']
  z = parameters[parameters['Parameter_ID']=='WH3Z'].iloc[0]['Value']

  warehouses.append(Warehouse(3, x,y,z, 5))





0.0 0.0 0.0


In [ ]:
ch = 'A'
temp_list = [1,1,1,4,5]
for i in range(26):
    temp = parameters[parameters['Type']=='Recharge'+str(i+1)]
    if(len(temp)==0):
        break
    x = temp[temp['Parameter_ID'] == ch + 'X1'].Value.iloc[0]
    y = temp[temp['Parameter_ID'] == ch + 'Y1'].Value.iloc[0]
    z = 0
    ch = chr(ord(ch) + 1)
    v = vertex(x,y,z)
    time_dict={}
    for j in range(max_time): time_dict[j]=0
    charging_stations.append(Charging_station(i+1,x,y,z,temp_list[i],3,time_dict))
print(len(charging_stations))

2


In [ ]:
pay_load_weight = [5,6,7,8,9,10]
pay_load_vol = [200,500,1000,2000,3000,5000]
base_weight = [2,2.5,3,3.5,4,5]
battery_cap = [2000,2500,3000,4000,5000,10000]
max_slots = [1,1,2,2,2,4]
fixed_cost = [10,15,20,20,30,50]
main_cost = [5,8,13,15,20,25]
Drones = []
for i in range(6):
    temp = parameters[parameters['Type']=='Drone'+str(i+1)]
    p=temp[temp['Parameter_ID'].str.startswith('P')]['Value'].iloc[0]
    q=temp[temp['Parameter_ID'].str.startswith('Q')]['Value'].iloc[0]
    a=temp[temp['Parameter_ID'].str.startswith('A')]['Value'].iloc[0]
    b=temp[temp['Parameter_ID'].str.startswith('B')]['Value'].iloc[0]
    c=temp[temp['Parameter_ID'].str.startswith('C')]['Value'].iloc[0]
    num=temp[temp['Parameter_ID'].str.startswith('DT')]['Value'].iloc[0]
    drone = Drone(i,max_speed,p,q,0,0,0,a,b,c,pay_load_vol[i],pay_load_weight[i],battery_cap[i],fixed_cost[i],main_cost[i],max_slots[i],base_weight[i],num)
    Drones.append(drone)
print(Drones[0].C)

0.01


In [ ]:
items.append(Item(1,1,5,8,5))
items.append(Item(2,6,5,10,8))
items.append(Item(3,4,5,10,15))
items.append(Item(4,2,15,10,8))
items.append(Item(5,5,20,15,10))

In [ ]:
for i in range(len(demand)):
    w_no=int(str(demand.loc[i,"WH"])[-1])
    i_no=int(str(demand.loc[i,"Item"])[-1])
    d_no=int(str(demand.loc[i,"Day"])[-1])
    x=demand.loc[i,"X"]
    y=demand.loc[i,"Y"]
    z=demand.loc[i,"Z"]
    starttime=demand.loc[i,"DeliveryFrom"]
    endtime=demand.loc[i,"DeliveryTo"]
    total_cities.append(City(i,x,y,z,items[i_no-1],starttime,endtime,warehouses[w_no-1],d_no))

In [ ]:
def min_distance_func(payload,city1,city2,drone,init_time):
  f=payload/drone.payload_capacity_weight
  time,dist,h,path=city1.location.min_time(city2.location,drone.velocities(f),init_time)
  # print(dist,h)
  energy_consumed=(payload+drone.base_weight)*((drone.A)+drone.B*dist+drone.C*h)
  fin_path=[]
  i=len(path)-1
  j=len(path)-2
  while (i>0):
    pr=path[i][1]
    j=i-1
    while j>=0:
      if pr==path[j][1]:
        j-=1
      else:
        break
    if(j==-1):
      j=0
    # print(i,j,path[i][1],path[j][1])
    t=path[j][1]-path[i][1]
    
    d=path[j][0].euc_distance(path[i][0])
    ei=(payload+drone.base_weight)*((drone.A)+drone.B*drone.velocities(f)[0]+drone.C*max(0,(path[j][0].cordinate[2]-path[i][0].cordinate[2])))
    # if path[i][0].cordinate[2]!=path[j][0].cordinate[2]:
    #   ei+=(payload+drone.base_weight)*((drone.A)+drone.B*drone.velocities(f)[2]+)
    timess=np.linspace(path[i][1],path[j][1],t+1)
    cordis=np.linspace(path[i][0].cordinate,path[j][0].cordinate,t+1)
    for k in range(t):
      fin_path.append((vertex(cordis[k][0],cordis[k][1],cordis[k][2]),timess[k],ei))
    i=j

  return energy_consumed,time,fin_path


In [ ]:

def check_order(path,drone,cost_c):            ##cost_c is refuelling cost
  
  chk=1
  initial_energy = drone.battery_capacity
  max_slots = drone.max_slots
  max_payload = drone.payload_capacity_weight     #max_payload_weight_can_carry
  max_volume = drone.payload_capacity_vol                    #max_volume
  
  

  curr_index_for_wh1=-1
  curr_index_for_wh2=-1
  curr_index_for_wh3=-1
  pickup_index_for_path=[]
  dynamic_payload=np.zeros(len(path))           #initializing_all_the_payload_volume and slot to be zeros
  dynamic_volume=np.zeros(len(path))
  dynamic_slot=np.zeros(len(path))

  

  for i in range(len(path)):                      #checking payload capacity, volume ,slot

    if(isinstance(path[i],Warehouse)):                              ##if its a ware_house
      if(path[i].id==1):
        pickup_index_for_path.append(path[i])
        curr_index_for_wh1=i
      elif(path[i].id==2):
        pickup_index_for_path.append(path[i])
        curr_index_for_wh2=i
      elif(path[i].id==3):
        pickup_index_for_path.append(path[i])
        curr_index_for_wh3=i

    elif(isinstance(path[i],City)):                                   ##if its a city
      if(path[i].warehouse.id==1):                                        #if demand is it warehouse 1
        pickup_index_for_path.append(curr_index_for_wh1)
        dynamic_payload[curr_index_for_wh1]+=path[i].item.weight
        dynamic_volume[curr_index_for_wh1]+=path[i].item.volume
        dynamic_slot[curr_index_for_wh1]+=1
        if(curr_index_for_wh1==-1 or dynamic_payload[curr_index_for_wh1]>max_payload or dynamic_volume[curr_index_for_wh1]>max_volume or dynamic_slot[curr_index_for_wh1]>max_slots ):
          chk=-1
        

      elif(path[i].warehouse.id==2):                                       #if demand is it warehouse 2
        pickup_index_for_path.append(curr_index_for_wh2)
        dynamic_payload[curr_index_for_wh2]+=path[i].item.weight
        dynamic_volume[curr_index_for_wh2]+=path[i].item.volume
        dynamic_slot[curr_index_for_wh2]+=1
        if(curr_index_for_wh2==-1 or dynamic_payload[curr_index_for_wh2]>max_payload or dynamic_volume[curr_index_for_wh2]>max_volume or dynamic_slot[curr_index_for_wh2]>max_slots ):
          chk=-1
       

      elif(path[i].warehouse.id==3):                                       #if demand is it warehouse 3
        pickup_index_for_path.append(curr_index_for_wh3)
        dynamic_payload[curr_index_for_wh3]+=path[i].item.weight
        dynamic_volume[curr_index_for_wh3]+=path[i].item.volume
        dynamic_slot[curr_index_for_wh3]+=1
        if(curr_index_for_wh3==-1 or dynamic_payload[curr_index_for_wh3]>max_payload or dynamic_volume[curr_index_for_wh3]>max_volume or dynamic_slot[curr_index_for_wh3]>max_slots ):
          chk=-1
       

    else:
      pickup_index_for_path.append(path[i])

  print("printing pick_up_index and dyamic payload,volume,slot \n")
  print(pickup_index_for_path,'\n',dynamic_payload,'\n',dynamic_volume,'\n',dynamic_slot,'\n')
  print("printing maximum payload,volume,slot of drone,maximum eergy")
  print(max_payload, max_volume, max_slots,initial_energy)

  if(chk==-1):
    return -1,-1,-1,-1,-1,-1       # returning is it possible,energy used,time of recharge*current*cost
  













  ################################# 2nd part
  curr_energy=initial_energy
  curr_payload=0
  curr_time=0                                #assuming time from 0s to 4*3600s

  #values_to_be_returned
  total_energy_used=0
  total_time_of_flight=0
  total_recharging_time_current_product=0
  total_variable_cost=0
  recharge_time=0  

  battery_status_for_every_visit_arrival=[]            ##for_reaching_every_node
  battery_status_for_every_visit_leaving=[]
  time_stamp_arrival=[]                                #for_reaching_every_node
  time_stamp_leaving=[]
  payload_each_node=[]

  for i in range(len(path)-1):

    if(curr_energy<0):
      chk=-1
    
    if(chk==-1):
      break

    battery_status_for_every_visit_arrival.append(curr_energy)
    time_stamp_arrival.append(curr_time)  
      
    

    if(isinstance(path[i],Warehouse)):                                                                        #if current stop is a warehouse
      if((dynamic_payload[i]>0) or (curr_payload>0)):
        curr_time+=180
      curr_payload=dynamic_payload[i]

    if(isinstance(path[i],City)): 
      if(path[i].delivery_failure==0):
          curr_payload=curr_payload-path[i].item.weight
     
    payload_each_node.append(curr_payload)


    #OR_team
    #payload given to function does not include base weight additionally
    # initVelocity = [0, 0]           #initial [horizontal, vertical] velocities
    # print(" curent energy is ",curr_energy," curr time is ",curr_time)
    # time_of_flight,final_energy,final_velocity,var_cost=pairOfNodes(curr_payload,path[i].location.cordinate,path[i+1].location.cordinate,curr_energy,initVelocity,50,drone,1000000)     #####function call to find optimal energy
    # print(" final energy is ",final_energy,"time_of flight is ",time_of_flight,'\n')
    


    #japesh_start
    print(" curent energy is ",curr_energy," curr time is ",curr_time)
    energy_consumed,time_of_flight,path  = min_distance_func(curr_payload,path[i],path[i+1],drone,0)
    var_cost=time_of_flight/3600*drone.var_cost
    final_energy=curr_energy-energy_consumed
    print(" final energy is ",final_energy,"time_of flight is ",time_of_flight,'\n')
    

    #japesh_end

    total_energy_used+=curr_energy-final_energy
    total_time_of_flight+=time_of_flight
    total_variable_cost+=var_cost

    

    if(isinstance(path[i],Warehouse)):
      recharge_time=0
      if((isinstance(path[i+1],City)) and (curr_time+time_of_flight<path[i+1].starttime) and curr_energy<initial_energy):                       #next stop is city
        recharge_time=min(path[i+1].starttime-curr_time-time_of_flight,((initial_energy-curr_energy)/(path[i].amp*1000))*3600)
        recharge_time=recharge_time/3600
        total_recharging_time_current_product+=recharge_time*path[i].amp*cost_c*3600                                 #per sec cost taken
      if(final_energy<0):
        recharge_time=abs(final_energy/(path[i].amp*1000))
        recharge_time=recharge_time/3600
        total_recharging_time_current_product+=recharge_time*path[i].amp*cost_c*3600                                 #per sec cost taken
      
      curr_time+=abs(recharge_time*3600)
      time_stamp_leaving.append(curr_time)
      curr_time+=time_of_flight
      battery_status_for_every_visit_leaving.append(curr_energy+path[i].amp*1000*recharge_time)
      curr_energy=final_energy+path[i].amp*1000*recharge_time
      

        


    if(isinstance(path[i],Charging_station)):
      recharge_time=0
      if((isinstance(path[i+1],City)) and (curr_time+time_of_flight<path[i+1].starttime) and curr_energy<initial_energy):                       #next stop is city
        recharge_time=min(path[i+1].starttime-curr_time-time_of_flight,((initial_energy-curr_energy)/(path[i].amp*1000))*3600)
        recharge_time=recharge_time/3600
        total_recharging_time_current_product+=recharge_time*path[i].amp*cost_c*3600                                 #per sec cost taken
      if(final_energy<0):
        recharge_time=abs(final_energy/(path[i].amp*1000))
        recharge_time=recharge_time/3600
        total_recharging_time_current_product+=recharge_time*path[i].amp*cost_c*3600                                 #per sec cost taken
      
      curr_time+=abs(recharge_time*3600)
      time_stamp_leaving.append(curr_time)
      curr_time+=time_of_flight
      battery_status_for_every_visit_leaving.append(curr_energy+path[i].amp*1000*recharge_time)
      curr_energy=final_energy+path[i].amp*1000*recharge_time



    if(isinstance(path[i],City)):                                                                            #if its a city
      print("delivery start and end time ",path[i].starttime," end time is  ",path[i].endtime,'\n')
      if(curr_time+180>path[i].endtime):
        chk=-1
      else:
        curr_time=np.maximum(curr_time+180,path[i].starttime+180)
        time_stamp_leaving.append(curr_time)
        curr_time+=time_of_flight
        battery_status_for_every_visit_leaving.append(curr_energy)
        curr_energy=final_energy
        
        

    


   




    
    

   
        


    
        

        
      

    
  time_stamp_arrival.append(curr_time)
  payload_each_node.append(curr_payload)
  battery_status_for_every_visit_arrival.append(curr_energy)
  if(curr_energy<0):
      chk=-1
  print("battery energy in arrival mAhr ",battery_status_for_every_visit_arrival,'\n',"battery energy in leaving mAhr ",battery_status_for_every_visit_leaving,'\n')
  print(" time stamp arrival ",time_stamp_arrival,'\n',"time stamp leaving",time_stamp_leaving,'\n')
  if(chk==-1):
     return -1,-1,-1,-1,-1,-1       
   
  return(1,total_energy_used,total_time_of_flight,total_recharging_time_current_product,total_variable_cost,payload_each_node)   # returning is it possible,energy used,final recharging product

        

In [ ]:
print(len(warehouses))
print(warehouses[0].id)
print(len(Drones))
cities=total_cities
for i in range(6):
  print(i, cities[i].warehouse.id,cities[i].starttime,cities[i].endtime)

print("for drones",'\n')
for i in range(len(Drones)):
  print(i,  Drones[i].id,Drones[i].battery_capacity, Drones[i].payload_capacity_weight, Drones[i].payload_capacity_vol)


1
1
6
0 1 600.0 3600.0
1 1 900.0 1800.0
2 1 1800.0 3600.0
3 1 2700.0 3600.0
4 1 7200.0 9000.0
5 1 12600.0 14400.0
for drones 

0 0 2000 5 200
1 1 2500 6 500
2 2 3000 7 1000
3 3 4000 8 2000
4 4 5000 9 3000
5 5 10000 10 5000


In [ ]:
cities.sort()
for i in range(len(cities)):
  print(cities[i].id," ",cities[i].starttime," ",cities[i].endtime)
Drones[0].max_slots

12   900.0   1500.0
37   900.0   1500.0
1   900.0   1800.0
22   600.0   1800.0
0   600.0   3600.0
2   1800.0   3600.0
3   2700.0   3600.0
6   1800.0   3600.0
11   1800.0   3600.0
15   1800.0   3600.0
27   1800.0   3600.0
36   1800.0   3600.0
39   1800.0   3600.0
40   2700.0   3600.0
17   4200.0   4800.0
21   4200.0   4800.0
28   4200.0   4800.0
32   4200.0   4800.0
43   4200.0   4800.0
8   3600.0   5400.0
33   3600.0   5400.0
44   3600.0   5400.0
7   5400.0   6300.0
23   5100.0   6600.0
4   7200.0   9000.0
10   3600.0   9000.0
35   3600.0   9000.0
41   7200.0   9000.0
42   5400.0   9000.0
46   7200.0   9000.0
16   7800.0   10200.0
9   3600.0   10800.0
34   3600.0   10800.0
45   3600.0   10800.0
25   10200.0   12000.0
47   7200.0   12600.0
5   12600.0   14400.0
13   11700.0   14400.0
14   8700.0   14400.0
18   9000.0   14400.0
19   13200.0   14400.0
20   12600.0   14400.0
24   13800.0   14400.0
26   13200.0   14400.0
29   9000.0   14400.0
30   13200.0   14400.0
31   12600.0   14400.0
38

1

In [ ]:

for i in range(len(Drones)):
  print(Drones[i].payload_capacity_weight,Drones[i].payload_capacity_vol)


5 200
6 500
7 1000
8 2000
9 3000
10 5000


In [ ]:

D1=[[] for i in range(int(Drones[0].number_of_drones))]
D2=[[] for i in range(int(Drones[1].number_of_drones))]
D3=[[] for i in range(int(Drones[2].number_of_drones))]
D4=[[] for i in range(int(Drones[3].number_of_drones))]
D5=[[] for i in range(int(Drones[4].number_of_drones))]
D6=[[] for i in range(int(Drones[5].number_of_drones))]
j=0
k=[0 for i in range(6)]
#D1[k].append(6)
nd=[0 for i in range(7)]
nd[6]=2000

for i in range(len(cities)):
  for j in range(6):
    a=Drones[j].payload_capacity_weight
    b=Drones[j].payload_capacity_vol
    if(a>=cities[i].item.weight and b>=cities[i].item.volume and nd[j]<=nd[j+1]):
      print(i,j,k,'\n')

      if(j==0):
        if(k[j]>=int(Drones[0].number_of_drones)):
          k[j]=0
        D1[k[j]].append(cities[i].id)
        nd[j]=len(D1[0])
        k[j]+=1
        break

      if(j==1):
        if(k[j]>=int(Drones[1].number_of_drones)):
          k[j]=0
        D2[k[j]].append(cities[i].id)
        nd[j]=len(D2[0])
        k[j]+=1
        break

      if(j==2):
        if(k[j]>=int(Drones[2].number_of_drones)):
          k[j]=0
        D3[k[j]].append(cities[i].id)
        nd[j]=len(D3[0])
        k[j]+=1
        break

      if(j==3):
        if(k[j]>=int(Drones[3].number_of_drones)):
          k[j]=0
        D4[k[j]].append(cities[i].id)
        nd[j]=len(D4[0])
        k[j]+=1
        break
      
      if(j==4):
        if(k[j]>=int(Drones[4].number_of_drones)):
          k[j]=0
        D5[k[j]].append(cities[i].id)
        nd[j]=len(D5[0])
        k[j]+=1
        break

      if(j==5):  
        if(k[j]>=int(Drones[5].number_of_drones)):
          k[j]=0
        D6[k[j]].append(cities[i].id)
        nd[j]=len(D6[0])
        k[j]+=1
        break







print(D1)
print(D2)
print(D3)
print(D4)
print(D5)
print(D6)
print(len(D1[0]))




0 0 [0, 0, 0, 0, 0, 0] 

1 1 [1, 0, 0, 0, 0, 0] 

2 2 [1, 1, 0, 0, 0, 0] 

3 3 [1, 1, 1, 0, 0, 0] 

4 0 [1, 1, 1, 1, 0, 0] 

5 2 [2, 1, 1, 1, 0, 0] 

6 4 [2, 1, 2, 1, 0, 0] 

7 5 [2, 1, 2, 1, 1, 0] 

8 1 [2, 1, 2, 1, 1, 1] 

9 1 [2, 2, 2, 1, 1, 1] 

10 2 [2, 3, 2, 1, 1, 1] 

11 1 [2, 3, 3, 1, 1, 1] 

12 4 [2, 4, 3, 1, 1, 1] 

13 3 [2, 4, 3, 1, 2, 1] 

14 0 [2, 4, 3, 2, 2, 1] 

15 4 [3, 4, 3, 2, 2, 1] 

16 2 [3, 4, 3, 2, 3, 1] 

17 3 [3, 4, 4, 2, 3, 1] 

18 1 [3, 4, 4, 3, 3, 1] 

19 1 [3, 5, 4, 3, 3, 1] 

20 2 [3, 1, 4, 3, 3, 1] 

21 0 [3, 1, 5, 3, 3, 1] 

22 3 [4, 1, 5, 3, 3, 1] 

23 4 [4, 1, 5, 4, 3, 1] 

24 2 [4, 1, 5, 4, 4, 1] 

25 2 [4, 1, 6, 4, 4, 1] 

26 0 [4, 1, 7, 4, 4, 1] 

27 4 [1, 1, 7, 4, 4, 1] 

28 0 [1, 1, 7, 4, 5, 1] 

29 2 [2, 1, 7, 4, 5, 1] 

30 3 [2, 1, 8, 4, 5, 1] 

31 0 [2, 1, 8, 5, 5, 1] 

32 3 [3, 1, 8, 5, 5, 1] 

33 4 [3, 1, 8, 6, 5, 1] 

34 2 [3, 1, 8, 6, 1, 1] 

35 0 [3, 1, 1, 6, 1, 1] 

36 3 [4, 1, 1, 6, 1, 1] 

37 0 [4, 1, 1, 7, 1, 1] 

38 1 [1, 1, 1, 7, 1, 1

In [ ]:

D1=[[] for i in range(int(Drones[0].number_of_drones))]
D2=[[] for i in range(int(Drones[1].number_of_drones))]
D3=[[] for i in range(int(Drones[2].number_of_drones))]
D4=[[] for i in range(int(Drones[3].number_of_drones))]
D5=[[] for i in range(int(Drones[4].number_of_drones))]
D6=[[] for i in range(int(Drones[5].number_of_drones))]
j=0
k=[0 for i in range(6)]
#D1[k].append(6)
nd=[0 for i in range(7)]
nd[6]=2000

for i in range(len(cities)):
  for j in range(6):
    a=Drones[j].payload_capacity_weight
    b=Drones[j].payload_capacity_vol
    if(a>=cities[i].item.weight and b>=cities[i].item.volume and nd[j]<=nd[j+1]):
      print(i,j,k,'\n')

      if(j==0):
        if(k[j]>=int(Drones[0].number_of_drones)):
          k[j]=0
        D1[k[j]].append(cities[i])
        nd[j]=len(D1[0])
        k[j]+=1
        break

      if(j==1):
        if(k[j]>=int(Drones[1].number_of_drones)):
          k[j]=0
        D2[k[j]].append(cities[i])
        nd[j]=len(D2[0])
        k[j]+=1
        break

      if(j==2):
        if(k[j]>=int(Drones[2].number_of_drones)):
          k[j]=0
        D3[k[j]].append(cities[i])
        nd[j]=len(D3[0])
        k[j]+=1
        break

      if(j==3):
        if(k[j]>=int(Drones[3].number_of_drones)):
          k[j]=0
        D4[k[j]].append(cities[i])
        nd[j]=len(D4[0])
        k[j]+=1
        break
      
      if(j==4):
        if(k[j]>=int(Drones[4].number_of_drones)):
          k[j]=0
        D5[k[j]].append(cities[i])
        nd[j]=len(D5[0])
        k[j]+=1
        break

      if(j==5):  
        if(k[j]>=int(Drones[5].number_of_drones)):
          k[j]=0
        D6[k[j]].append(cities[i])
        nd[j]=len(D6[0])
        k[j]+=1
        break







print(D1)
print(D2)
print(D3)
print(D4)
print(D5)
print(D6)
print(len(D1[0]))




0 0 [0, 0, 0, 0, 0, 0] 

1 1 [1, 0, 0, 0, 0, 0] 

2 2 [1, 1, 0, 0, 0, 0] 

3 3 [1, 1, 1, 0, 0, 0] 

4 0 [1, 1, 1, 1, 0, 0] 

5 2 [2, 1, 1, 1, 0, 0] 

6 4 [2, 1, 2, 1, 0, 0] 

7 5 [2, 1, 2, 1, 1, 0] 

8 1 [2, 1, 2, 1, 1, 1] 

9 1 [2, 2, 2, 1, 1, 1] 

10 2 [2, 3, 2, 1, 1, 1] 

11 1 [2, 3, 3, 1, 1, 1] 

12 4 [2, 4, 3, 1, 1, 1] 

13 3 [2, 4, 3, 1, 2, 1] 

14 0 [2, 4, 3, 2, 2, 1] 

15 4 [3, 4, 3, 2, 2, 1] 

16 2 [3, 4, 3, 2, 3, 1] 

17 3 [3, 4, 4, 2, 3, 1] 

18 1 [3, 4, 4, 3, 3, 1] 

19 1 [3, 5, 4, 3, 3, 1] 

20 2 [3, 1, 4, 3, 3, 1] 

21 0 [3, 1, 5, 3, 3, 1] 

22 3 [4, 1, 5, 3, 3, 1] 

23 4 [4, 1, 5, 4, 3, 1] 

24 2 [4, 1, 5, 4, 4, 1] 

25 2 [4, 1, 6, 4, 4, 1] 

26 0 [4, 1, 7, 4, 4, 1] 

27 4 [1, 1, 7, 4, 4, 1] 

28 0 [1, 1, 7, 4, 5, 1] 

29 2 [2, 1, 7, 4, 5, 1] 

30 3 [2, 1, 8, 4, 5, 1] 

31 0 [2, 1, 8, 5, 5, 1] 

32 3 [3, 1, 8, 5, 5, 1] 

33 4 [3, 1, 8, 6, 5, 1] 

34 2 [3, 1, 8, 6, 1, 1] 

35 0 [3, 1, 1, 6, 1, 1] 

36 3 [4, 1, 1, 6, 1, 1] 

37 0 [4, 1, 1, 7, 1, 1] 

38 1 [1, 1, 1, 7, 1, 1

In [ ]:
print(len(D1[0]))                       ##first drone
for i in range(len(D1)):
  iter=1
  lis=[]
  lis=D1[i].copy()
  mm=len(lis)
  print(lis)
  lis.insert(0,warehouses[0])
  p=0
  lis.append(warehouses[0])
  q=len(lis)-1
  print(i,'\n')
  chk, energy, flight_time, recharging_cost, var_cost,pay_load_order= check_order(lis, Drones[0], 0.01)
  if(chk==-1):
    if(iter==1 and mm>=2):
      lis.insert(2,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost,pay_load_order= check_order(lis, Drones[0], 0.01)
    
  
  if(chk==-1):
    if(iter==2 and mm>=3):
      lis.insert(4,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost,pay_load_order= check_order(lis, Drones[0], 0.01)
 
  print("size of list",len(lis))    
  print(len(lis))
  print(chk,energy, flight_time, recharging_cost, var_cost)
  print(i," NEXT LIST UPLOADING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'\n \n \n \n  \n \n \n \n \n')

  print(len)


3
[<__main__.City object at 0x7fc7913ce990>, <__main__.City object at 0x7fc791437750>, <__main__.City object at 0x7fc7917c1e10>]
0 

printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fc791438690>, 0, 0, 0, <__main__.Warehouse object at 0x7fc791438690>] 
 [3. 0. 0. 0. 0.] 
 [600.   0.   0.   0.   0.] 
 [3. 0. 0. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
5 200 1 2000
printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fc791438690>, 0, <__main__.Warehouse object at 0x7fc791438690>, 2, 2, <__main__.Warehouse object at 0x7fc791438690>] 
 [1. 0. 2. 0. 0. 0.] 
 [200.   0. 400.   0.   0.   0.] 
 [1. 0. 2. 0. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
5 200 1 2000
printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fc791438690>, 0, <__main__.Warehouse object at 0x7fc791438690>, 2, <__main__.Warehouse object at 0x7fc791438690>, 4

AttributeError: ignored

In [ ]:
print(len(D2[0]))                                         ##second drone
for i in range(len(D2)):
  iter=1
  lis=[]
  lis=D2[i].copy()
  mm=len(lis)
  print(lis)
  lis.insert(0,warehouses[0])
  p=0
  lis.append(warehouses[0])
  q=len(lis)-1
  print(i,'\n')
  chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[1], 0.01)
  if(chk==-1):
    if(iter==1 and mm>=2):
      lis.insert(2,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[1], 0.01)
    
  
  if(chk==-1):
    if(iter==2 and mm>=3):
      lis.insert(4,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[1], 0.01)
 
  print("size of list",len(lis))    
  print(len(lis))
  print(chk,energy, flight_time, recharging_cost, var_cost)
  print(i," NEXT LIST UPLOADING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'\n \n \n \n  \n \n \n \n \n')


3
[<__main__.City object at 0x7fe6cd273cd0>, <__main__.City object at 0x7fe6cb73bf90>, <__main__.City object at 0x7fe6cd273d10>]
0 

printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, 0, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [13.  0.  0.  0.  0.] 
 [1000.    0.    0.    0.    0.] 
 [3. 0. 0. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
6 500 1 2500
printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>, 2, 2, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [ 1.  0. 12.  0.  0.  0.] 
 [200.   0. 800.   0.   0.   0.] 
 [1. 0. 2. 0. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
6 500 1 2500
printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>, 2, <__main__.Warehouse object at 0x

In [ ]:
print(len(D3[0]))                                        ##third drone
for i in range(len(D3)):
  iter=1
  lis=[]
  lis=D3[i].copy()
  mm=len(lis)
  print(lis)
  lis.insert(0,warehouses[0])
  p=0
  lis.append(warehouses[0])
  q=len(lis)-1
  print(i,'\n')
  chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[2], 0.01)
  if(chk==-1):
    if(iter==1 and mm>=2):
      lis.insert(2,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[2], 0.01)
    
  
  if(chk==-1):
    if(iter==2 and mm>=3):
      lis.insert(4,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[2], 0.01)
 
  print("size of list",len(lis))    
  print(len(lis))
  print(chk,energy, flight_time, recharging_cost, var_cost)
  print(i," NEXT LIST UPLOADING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'\n \n \n \n  \n \n \n \n \n')


2
[<__main__.City object at 0x7fe6d0406910>, <__main__.City object at 0x7fe6cb73b890>]
0 

printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [12.  0.  0.  0.] 
 [800.   0.   0.   0.] 
 [2. 0. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
7 1000 2 3000
printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>, 2, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [6. 0. 6. 0. 0.] 
 [400.   0. 400.   0.   0.] 
 [1. 0. 1. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
7 1000 2 3000
 curent energy is  3000  curr time is  180
 final energy is  2191.449732883071 time_of flight is  368.1529878665661 

 curent energy is  2191.449732883071  curr time is  548.152987866566
 final energy is  1935.8463727357125 time_of flight is  315.5597038856272 

delive

In [ ]:
print(len(D4[0]))                                                 ##fourth drone
for i in range(len(D4)):
  iter=1
  lis=[]
  lis=D4[i].copy()
  mm=len(lis)
  print(lis)
  lis.insert(0,warehouses[0])
  p=0
  lis.append(warehouses[0])
  q=len(lis)-1
  print(i,'\n')
  chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[3], 0.01)
  if(chk==-1):
    if(iter==1 and mm>=2):
      lis.insert(2,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[3], 0.01)
    
  
  if(chk==-1):
    if(iter==2 and mm>=3):
      lis.insert(4,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[3], 0.01)
 
  print("size of list",len(lis))    
  print(len(lis))
  print(chk,energy, flight_time, recharging_cost, var_cost)
  print(i," NEXT LIST UPLOADING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'\n \n \n \n  \n \n \n \n \n')


2
[<__main__.City object at 0x7fe6cb73b250>, <__main__.City object at 0x7fe6cb75bed0>]
0 

printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [4. 0. 0. 0.] 
 [2400.    0.    0.    0.] 
 [2. 0. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
8 2000 2 4000
printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>, 2, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [2. 0. 2. 0. 0.] 
 [1200.    0. 1200.    0.    0.] 
 [1. 0. 1. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
8 2000 2 4000
 curent energy is  4000  curr time is  180
 final energy is  3167.5071143240357 time_of flight is  304.78029153096975 

 curent energy is  3167.5071143240357  curr time is  484.78029153096975
 final energy is  2647.4757418993186 time_of flight is  285.731523310283

In [ ]:
print(len(D5[0]))                                                 ##fifth drone
for i in range(len(D5)):
  iter=1
  lis=[]
  lis=D5[i].copy()
  mm=len(lis)
  print(lis)
  lis.insert(0,warehouses[0])
  p=0
  lis.append(warehouses[0])
  q=len(lis)-1
  print(i,'\n')
  chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[4], 0.01)
  if(chk==-1):
    if(iter==1 and mm>=2):
      lis.insert(2,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[4], 0.01)
    
  
  if(chk==-1):
    if(iter==2 and mm>=3):
      lis.insert(4,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[4], 0.01)
 
  print("size of list",len(lis))    
  print(len(lis))
  print(chk,energy, flight_time, recharging_cost, var_cost)
  print(i," NEXT LIST UPLOADING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'\n \n \n \n  \n \n \n \n \n')


2
[<__main__.City object at 0x7fe6cb73bb50>, <__main__.City object at 0x7fe6cb75bdd0>]
0 

printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [7. 0. 0. 0.] 
 [4200.    0.    0.    0.] 
 [2. 0. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
9 3000 2 5000
printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>, 2, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [2. 0. 5. 0. 0.] 
 [1200.    0. 3000.    0.    0.] 
 [1. 0. 1. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
9 3000 2 5000
 curent energy is  5000  curr time is  180
 final energy is  4255.150187781683 time_of flight is  242.16881199282037 

 curent energy is  4255.150187781683  curr time is  422.16881199282034
 final energy is  3770.2744108805023 time_of flight is  228.7149891043302 

In [ ]:
print(len(D6[0]))                                                 ##sixth drone
for i in range(len(D6)):
  iter=1
  lis=[]
  lis=D6[i].copy()
  mm=len(lis)
  print(lis)
  lis.insert(0,warehouses[0])
  p=0
  lis.append(warehouses[0])
  q=len(lis)-1
  print(i,'\n')
  chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[5], 0.01)
  if(chk==-1):
    if(iter==1 and mm>=2):
      lis.insert(2,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[5], 0.01)
    
  
  if(chk==-1):
    if(iter==2 and mm>=3):
      lis.insert(4,warehouses[0])
      iter+=1
      chk, energy, flight_time, recharging_cost, var_cost= check_order(lis, Drones[5], 0.01)
 
  print("size of list",len(lis))    
  print(len(lis))
  print(chk,energy, flight_time, recharging_cost, var_cost)
  print(i," NEXT LIST UPLOADING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>",'\n \n \n \n  \n \n \n \n \n')


1
0 

printing pick_up_index and dyamic payload,volume,slot 

[<__main__.Warehouse object at 0x7fe6cb73bed0>, 0, <__main__.Warehouse object at 0x7fe6cb73bed0>] 
 [5. 0. 0.] 
 [3000.    0.    0.] 
 [1. 0. 0.] 

printing maximum payload,volume,slot of drone,maximum eergy
10 5000 4 10000
 curent energy is  10000  curr time is  180
 final energy is  8369.425596710933 time_of flight is  289.2680733252847 

 curent energy is  8369.425596710933  curr time is  469.2680733252847
 final energy is  7585.991231454953 time_of flight is  241.05672777107077 

delivery start and end time  1800.0  end time is   3600.0 

battery energy in arrival mAhr  [10000, 8369.425596710933, 7585.991231454953] 
 battery energy in leaving mAhr  [10000, 8369.425596710933] 

 time stamp arrival  [0, 469.2680733252847, 2221.0567277710707] 
 time stamp leaving [180, 1980.0] 

size of list 3
3
1 2414.008768545047 530.3248010963555 0 3.682811118724691
0  NEXT LIST UPLOADING>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>